In [4]:
from rdkit import Chem
import numpy as np

In [5]:
SMILES_CHAR = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'n', 'o', 'p', 'r', 's', 't', 'u',
               'A', 'B', 'C', 'G', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 
               'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
               '#', '%', '(', ')', '[', ']', '-', '+', '=', '.', '/', '\\', '@']

char_to_ind = {c : i for i, c in enumerate(SMILES_CHAR)}

In [6]:
def smiles_to_array(smiles: str, max_len: int) -> np.array:
    mol_array = np.zeros((max_len, len(SMILES_CHAR)))
    for i, c in enumerate(smiles):
        mol_array[i][char_to_ind[c]] = 1
    return mol_array

def array_to_smiles(mol_array: np.array) -> str:
    smiles = ''
    for l in mol_array:
        j = np.argmax(l)
        if l[j] == 0:
            break
        smiles += SMILES_CHAR[j]
    return smiles

In [ ]:
smiles = 'CCN(CC)C(=O)C1=CC=CC(=C1)C'
ar = smiles_to_array(smiles, 100)
array_to_smiles(ar)

In [11]:
import os
import rapidjson
import cv2
from sklearn.model_selection import train_test_split

def create_filename(num: int) -> str:
    return str(num) + '.png'

FOLDER_NAME = 'images'
OUTPUT_FILENAME = 'num_of_images.json'

with open(OUTPUT_FILENAME, 'r') as json_file:
    data = rapidjson.load(json_file)

max_len = max([len(datum['smiles']) for datum in data])
pictures = []
smileses = []
for datum in data:
    img = cv2.imread(
        os.path.join(FOLDER_NAME, create_filename(datum['image_num'])),
        cv2.IMREAD_GRAYSCALE
    )
    img = img.astype('float32')
    img /= 255
    pictures.append(img)
    smileses.append(np.concatenate(smiles_to_array(datum['smiles'], max_len)))
pictures = np.expand_dims(np.array(pictures), axis=3)
smileses = np.array(smileses)
print(pictures.shape)
print(smileses.shape)
p_train, p_test, s_train, s_test = train_test_split(pictures, smileses, test_size=0.2)

(2815, 128, 128, 1)
(2815, 1798)


In [12]:
import numpy as np
np.random.seed(123)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
import tensorflow as tf
from matplotlib import pyplot as plt

"""
(X_train, y_train), (X_test, y_test) = mnist.load_data()
plt.imshow(X_train[239], cmap='gray')
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
X_train = np.array([np.concatenate(x) for x in X_train])
X_test = np.array([np.concatenate(x) for x in X_test])
"""

model = Sequential()

#model.add(Dense(input_shape=(128 ** 2,), units=128, activation='relu'))
#model.add(Conv2D(4096, kernel_size=3, activation='relu'))
#model.add(Flatten())
#model.add(Dense(1798, activation='relu'))
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(128,128,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(1798,activation="relu"))

model.summary()

model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 16)      80        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 32)        2080      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 64)        8256      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)            

In [13]:
model.fit(p_train, s_train, 
          batch_size=32, epochs=20)

Epoch 1/20
71/71 [==============================] - 21s 281ms/step - loss: 140.7182 - accuracy: 0.0000e+00
Epoch 2/20
71/71 [==============================] - 20s 283ms/step - loss: 130.7624 - accuracy: 0.0417
Epoch 3/20
71/71 [==============================] - 21s 294ms/step - loss: 118.1093 - accuracy: 0.8490
Epoch 4/20
71/71 [==============================] - 20s 286ms/step - loss: 117.3926 - accuracy: 0.4578
Epoch 5/20
71/71 [==============================] - 20s 287ms/step - loss: 111.6104 - accuracy: 0.0000e+00
Epoch 6/20
71/71 [==============================] - 21s 289ms/step - loss: 110.1649 - accuracy: 0.0000e+00
Epoch 7/20
71/71 [==============================] - 21s 294ms/step - loss: 106.5800 - accuracy: 0.0000e+00
Epoch 8/20
71/71 [==============================] - 21s 290ms/step - loss: 104.7506 - accuracy: 0.0675
Epoch 9/20
71/71 [==============================] - 20s 284ms/step - loss: 104.3889 - accuracy: 0.0728
Epoch 10/20
71/71 [==============================] - 21s 

In [ ]:
model.evaluate(X_test, Y_test)